In [12]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.applications.inception_v3 import preprocess_input

# Necessary imports
import sys
sys.path.append('src/')
import utils, models, layers, metrics
from tensorflow.keras.layers import BatchNormalization, Conv2D, \
    Dense, UpSampling2D, GlobalMaxPool2D, GlobalAveragePooling2D, Flatten, Embedding, Concatenate
from utils import cast_img
from layers import BConv2D, LConv2D, SelfAttention, ResBlock, ConditionalBatchNorm,ConditionalDenseBatchNorm, Projection, Convolution, DenselyConnected, TransposedConvolution
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import datetime
from pathlib import Path
from PIL import Image


In [13]:
new_model = tf.keras.models.load_model('Models/20210625-141706/generator')#http://10.28.202.5:8888/tree/tensorflow/Samed/Models/20210625-141706/generator
#new_model.summary()

In [2]:
num_samples = 20
intraFID = metrics.IntraFID(input_shape=(299,299,3), num_samples=num_samples)
intraFID._load_data()
#for image_class in intraFID.image_chunks:
    #for image in image_class:
        #plt.figure()
        #plt.imshow((image + 1.0)*0.5)
        #plt.show()

In [ ]:
new_model = tf.keras.models.load_model('Models/20210625-141706/generator')#http://10.28.202.5:8888/tree/tensorflow/Samed/Models/20210625-141706/generator
#new_model.summary()

In [16]:
@tf.function
def generate(model, z,y):
    return model([z,y], training=False)

y_list = [tf.fill([10, 1], i) for i in range(10)]
z_fid = tf.random.normal((10, 10, 100))
fake_images = [generate(new_model,z,y2) for y2, z in zip(y_list, z_fid)]
i = 0
for image_class in fake_images:
    for image in image_class:
        image = np.cast(image + 1.0) * 127.5, 'uint8'))
        im = Image.fromarray(image)
        im.save("image"+str(i)+".jpeg")
        #plt.figure()
        #plt.imshow((image + 1.0)*0.5)
        #plt.show()
fid_classes, fid_scalar = intraFID.get_fid(fake_images)
print(fid_classes)

SyntaxError: invalid syntax (<ipython-input-16-8cc47496b2f8>, line 11)

In [3]:
def build_discriminator(hp):
    input_image = tf.keras.Input(shape=(32, 32, 3))
    input_label = tf.keras.Input(shape=(1,))
    
    xy = input_image
    
    model_type = hp.Choice("d_model_type", ["concat", "hidden_concat", "projection"])
    if model_type == "concat":
        with hp.conditional_scope("d_model_type", ["concat"]):
            y = Embedding(10, hp.Int("d_embedding_dim", 10, 100, step=10))(input_label)
            y = DenselyConnected(32*32, sn=hp.Boolean("d_concat_sn"))(y)
            y = tf.keras.activations.relu(y, alpha=hp.Float("d_lrelu_concat", 0.0, 0.3, step=0.1))
            y = tf.reshape(y, [-1, 32, 32, 1])
            xy = Concatenate()([input_image, y])
    d_concat_stage = hp.Int("d_concat_stage", 0, 2)   
    for i in range(3):

            xy = Convolution(128*(2**i), kernel_size=hp.Int("d_kernel_size_" + str(i), 3, 5), 
                             sn=hp.Boolean("d_spectral_conv_"+ str(i)), padding='same', strides=2)(xy)
            xy = tf.keras.activations.relu(xy, alpha=hp.Float("d_lrelu_conv_"+ str(i), 0.0, 0.3, step=0.1))
            if model_type == "hidden_concat":
                with hp.conditional_scope("d_model_type", ["hidden_concat"]):
                    if i == d_concat_stage:
                        dim =32//(2**(i+1))
                        y = Embedding(10, hp.Int("d_embedding_dim", 10, 100, step=10))(input_label)
                        y = DenselyConnected(dim*dim, sn=hp.Boolean("d_hidden_concat_sn_"+str(i)))(y)
                        y = tf.keras.activations.relu(y, alpha=hp.Float("d_lrelu_conv_"+ str(i), 0.0, 0.3, step=0.1))
                        y = tf.reshape(y, [-1, dim, dim, 1])
                        xy = Concatenate()([xy, y])
     
    xy = tf.keras.activations.relu(xy, alpha=hp.Float("d_lrelu_pred", 0.0, 0.3, step=0.1))
    if model_type == "projection":
            with hp.conditional_scope("d_model_type", ["projection"]):    
                outputs = Projection(xy, input_label, 10, sn=hp.Boolean("d_projection_sn"))
    if model_type == "concat" or model_type == "hidden_concat":
            with hp.conditional_scope("d_model_type", ["concat", "hidden_concat"]):  
                xy = GlobalAveragePooling2D()(xy)
                outputs = DenselyConnected(1, sn=hp.Boolean(model_type + "_pred_sn"))(xy)
    model = tf.keras.Model(inputs=[input_image,input_label], outputs=outputs, name='Discriminator')
    return model

def build_generator(hp):
        
        input_z = tf.keras.Input(shape=(hp.Int("g_z_dim",50, 300, step=10),))
        input_label = tf.keras.Input(shape=(1,))
        input_label_int = tf.cast(input_label, tf.int32)
        
        xy = DenselyConnected(4*4*hp.Int("g_z_dense", 10, 512), sn=hp.Boolean("g_densely_z_sn"))(input_z)
        xy = tf.keras.activations.relu(xy, alpha=hp.Float("g_lrelu_densely_z", 0.0, 0.3, step=0.1))
        xy = tf.reshape(xy, [-1, 4, 4, hp.get("g_z_dense")])
        
        
        #xy = x
        
        model_type = hp.Choice("g_model_type", ["concat", "conditional"])
        
        if model_type == "concat":
            with hp.conditional_scope("g_model_type", ["concat"]):
                y = Embedding(10, hp.Int("g_concat_embedding_dim", 10, 100, step=10))(input_label)
                y = DenselyConnected(4*4, sn=hp.Boolean("g_concat_sn"))(y)
                y = tf.keras.activations.relu(y, alpha=hp.Float("g_lrelu_concat", 0.0, 0.3, step=0.1))
                y = tf.reshape(y, [-1, 4, 4, 1])
                xy = Concatenate()([xy, y])
            
        if model_type == "conditional":
            with hp.conditional_scope("g_model_type", ["conditional"]):
                cbn_type = hp.Choice("g_CBNType", ["label", "linear"])
                if cbn_type == "linear":
                
                    y = Embedding(10, hp.Int("g_linear_embedding_dim", 10, 100, step=10))(input_label)
                    y = tf.reshape(y, [-1, hp.get("g_linear_embedding_dim")])
                    y = Concatenate()([input_z, y])
                
        
        for i in range(3):    
            
            xy = TransposedConvolution(512//pow(2,i), hp.Int("g_kernel_size_" + str(i), 3, 5), sn=hp.Boolean("g_spectral_conv" + str(i)), padding='same', kernel_initializer='glorot_normal', strides=2)(xy)
            
            if model_type == "conditional":
                if cbn_type == "label":
                    xy = ConditionalBatchNorm(num_classes=10)([xy, input_label_int])
                else:
                    xy = ConditionalDenseBatchNorm()([xy, y])
            if model_type == "concat":
                    xy = BatchNormalization()(xy)
                    
            xy = tf.keras.activations.relu(xy, alpha=hp.Float("g_lrelu_conv_" + str(i), 0.0, 0.3, step=0.1))

        outputs = Convolution(3, 3, sn=hp.Boolean("g_sn_output"), activation='tanh', padding='same')(xy)
        model = tf.keras.Model(inputs=[input_z, input_label], outputs=outputs, name='Generator')
        return model

    
class GAN(tf.keras.Model):
    def __init__(self, hp):
        super(GAN, self).__init__()
        self.g = build_generator(hp)
        self.d = build_discriminator(hp)

In [6]:
class MyTuner(kt.Tuner):
    def run_trial(self, trial, train_ds):
        hp = trial.hyperparameters

        # Hyperparameters can be added anywhere inside `run_trial`.
        # When the first trial is run, they will take on their default values.
        # Afterwards, they will be tuned by the `Oracle`.
        train_ds = train_ds.batch(hp.Int("batch_size", 64, 256, step=64, default=64)).prefetch(tf.data.experimental.AUTOTUNE)

        model = self.hypermodel.build(trial.hyperparameters)
        lr_d = hp.Float("learning_rate_d", 0.0001, 0.0004, sampling="log", default=0.0002)
        lr_g = hp.Float("learning_rate_g", 0.00005, 0.0002, sampling="log", default=0.00005)
        beta_1_d = hp.Float("beta1_d", 0.0, 0.5)
        beta_1_g = hp.Float("beta1_g", 0.0, 0.5)
        d_optimizer = tf.keras.optimizers.Adam(lr_d, beta_1=beta_1_d, beta_2=0.9)
        g_optimizer = tf.keras.optimizers.Adam(lr_g, beta_1=beta_1_g, beta_2=0.9)
        
        def discriminator_loss(real_output, fake_output): 
        #return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)
            return tf.reduce_mean(tf.nn.relu(1.0 - real_output)) + tf.reduce_mean(tf.nn.relu(1.0 + fake_output))

        def generator_loss(fake_output):
            return -tf.reduce_mean(fake_output)

        @tf.function
        def train_step_d(real_images, real_label):
        
            noise = tf.random.normal((hp.get("batch_size"), hp.get("g_z_dim")))
            label = tf.random.uniform(shape=(hp.get("batch_size"), 1), minval=0, maxval=10, dtype=tf.int64)
            with tf.GradientTape() as disc_tape:

                generated_images = model.g([noise, label],training=True)
                fake_logits = model.d([generated_images, label], training=True)
                real_logits = model.d([real_images, real_label], training=True)
                disc_loss = discriminator_loss(real_logits, fake_logits)
                #gp_loss = self.gradient_penalty(self.d, real_image, generated_image, real_label)
                #disc_loss += gp_loss * self.lambda_gp
            gradients_of_discriminator = disc_tape.gradient(disc_loss, model.d.trainable_variables)
            d_optimizer.apply_gradients(zip(gradients_of_discriminator, model.d.trainable_variables))
            gradient_norm = tf.linalg.global_norm(gradients_of_discriminator)
            return disc_loss, fake_logits, real_logits, gradient_norm
        
        @tf.function  # Training function
        def train_step_g():
            noise = tf.random.normal((hp.get("batch_size"), hp.get("g_z_dim")))  # latent noise vector
            label = tf.random.uniform(shape=(hp.get("batch_size"), 1), minval=0, maxval=10, dtype=tf.int64)
            with tf.GradientTape() as gen_tape:

                generated_images = model.g([noise, label], training=True)
                fake_logits = model.d([generated_images, label], training=True)
                gen_loss = generator_loss(fake_logits)
            gradients_of_generator = gen_tape.gradient(gen_loss, model.g.trainable_variables)
            g_optimizer.apply_gradients(zip(gradients_of_generator, model.g.trainable_variables))
            gradient_norm = tf.linalg.global_norm(gradients_of_generator)
            return gen_loss, gradient_norm
        
        @tf.function
        def generate(z,y):
            return model.g([z,y], training=False)
        # `self.on_epoch_end` reports results to the `Oracle` and saves the
        # current state of the Model. The other hooks called here only log values
        # for display but can also be overridden. For use cases where there is no
        # natural concept of epoch, you do not have to call any of these hooks. In
        # this case you should instead call `self.oracle.update_trial` and
        # `self.oracle.save_model` manually.
        s = tf.constant([0], dtype=tf.int64)
        z = tf.random.normal((10, hp.get("g_z_dim")))
        y = tf.reshape(tf.range(0, 10, delta=1, dtype=tf.int64), [10, 1])
        
        y_list = [tf.fill([num_samples, 1], i) for i in range(10)]
        
        current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        model_path = "Models/" + current_time
        Path(model_path).mkdir(parents=True, exist_ok=True)

        log_path = "Logs/" + current_time
        Path(log_path).mkdir(parents=True, exist_ok=True)
        writer = tf.summary.create_file_writer(log_path)
            
        template = 'Epoch: {}  Step: {}  G_loss: {}  D_loss {}'
        for epoch in range(100):

            for batch, label in train_ds:

                d_loss, fake_logits, real_logits, d_norm = train_step_d(batch, label)
                g_loss, g_norm = train_step_g()
                
                if s % 50 == 0:
                    print(template.format(epoch, s, g_loss, d_loss))

                
                    #image = model.g([z,y], training=False) 
                    image = generate(z,y)
                    #fake_images = [self.generate(z,y2) for y2 in y_list]
                    #fid_classes, fid_scalar = fid.get_fid(fake_images)
                    
                    with writer.as_default():
                        tf.summary.scalar("/1 Fake Output", tf.reduce_mean(fake_logits), step=s)
                        tf.summary.scalar("/1 Real Output", tf.reduce_mean(real_logits), step=s)
                        tf.summary.scalar("/2 G Loss", g_loss, step=s)
                        tf.summary.scalar("/2 D Loss", d_loss, step=s)
                        tf.summary.scalar("/3 D Gradient Norm", d_norm, step=s)
                        tf.summary.scalar("/3 G Gradient Norm", g_norm, step=s)
                        tf.summary.image("Generated Image", cast_img(image), step=s, max_outputs=10)
                            
                s += 1
            if epoch % 10 == 0 and epoch != 0:
                
                z_fid = tf.random.normal((10, num_samples, hp.get("g_z_dim")))
                fake_images = [generate(z_fi,y2) for y2, z_fi in zip(y_list, z_fid)]
                fid_classes, fid_scalar = intraFID.get_fid(fake_images)
                ssim_list = metrics.sample_ssim(model.g, 10, hp.get("g_z_dim"), n_samples=100)
                ssim_mean = np.mean(ssim_list)
                with writer.as_default():
                        #for count, ssim_class in enumerate(ssim_list):
                            #tf.summary.scalar("/4 MS-SSIM for Class {}".format(count), ssim_class, step=e)
                        tf.summary.scalar("/4 MS-SSIM Mean", ssim_mean, step=epoch)
                        tf.summary.scalar("/5 Mean FID", fid_scalar, step=epoch)
                        #for count, fid_class in enumerate(fid_classes):
                            #tf.summary.scalar("/5 Class FID {}".format(count), fid_class, step=e)
                            
                self.oracle.update_trial(trial.trial_id, {'fid': fid_scalar})
                self.save_model(trial.trial_id, model)
                #self.g.save(model_path + "/generator")
                #self.d.save(model_path + "/discriminator")
                #self.on_epoch_end(trial, model, epoch, logs={"loss": epoch_loss})
                #epoch_loss_metric.reset_states()


tuner = MyTuner(
    oracle=kt.oracles.BayesianOptimization(
        objective=kt.Objective("fid", "min"), max_trials=10
    ),
    hypermodel=GAN,
    directory="results",
    project_name="cifar10_custom_training",
)

INFO:tensorflow:Reloading Oracle from existing project results/cifar10_custom_training/oracle.json
INFO:tensorflow:Reloading Tuner from results/cifar10_custom_training/tuner0.json


In [ ]:
ds_train, ds_info = tfds.load('cifar10', split='train+test', shuffle_files=True, as_supervised=True, with_info=True)

def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    #image = tf.image.resize(image, [w_dim, h_dim])
    image = (tf.cast(image, tf.float32) / 128.) - 1.
    return image, label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)


tuner.search(train_ds=ds_train)



In [11]:
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model = tuner.get_best_models()[0]

{'g_z_dim': 260, 'g_z_dense': 184, 'g_densely_z_sn': False, 'g_lrelu_densely_z': 0.0, 'g_model_type': 'conditional', 'g_CBNType': 'linear', 'g_kernel_size_0': 4, 'g_spectral_conv0': True, 'g_lrelu_conv_0': 0.30000000000000004, 'g_kernel_size_1': 3, 'g_spectral_conv1': True, 'g_lrelu_conv_1': 0.2, 'g_kernel_size_2': 4, 'g_spectral_conv2': False, 'g_lrelu_conv_2': 0.2, 'g_sn_output': True, 'd_model_type': 'projection', 'd_kernel_size_0': 3, 'd_spectral_conv_0': False, 'd_lrelu_conv_0': 0.2, 'd_kernel_size_1': 4, 'd_spectral_conv_1': True, 'd_lrelu_conv_1': 0.30000000000000004, 'd_kernel_size_2': 3, 'd_spectral_conv_2': False, 'd_lrelu_conv_2': 0.1, 'd_lrelu_pred': 0.2, 'd_concat_stage': 1, 'batch_size': 128, 'learning_rate_d': 0.00022894833123711305, 'learning_rate_g': 0.00017284849450584436, 'beta1_d': 0.02713160345720983, 'beta1_g': 0.24735031833847654, 'g_linear_embedding_dim': 30, 'd_projection_sn': False}
